<a href="https://www.kaggle.com/code/tunguz/mnist-xgboost-baseline?scriptVersionId=96700944" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
col = ['pixel%d'%i for i in range(784)]

In [3]:
target = train[['label']].values
train.drop(['label'], axis=1, inplace=True)

In [4]:
features = train.columns

In [5]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train[features] = RS.transform(train)
test[features] = RS.transform(test)
del RS
gc.collect()
gc.collect()

CPU times: user 41.7 s, sys: 1min 24s, total: 2min 6s
Wall time: 2min 7s


0

In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(train, target, test_size=0.1, random_state=42)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

In [7]:
%%time

watchlist = [(dval, 'eval'), (dtrain, 'train')]

xgb_params = {'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.0014345301923969806,
 'alpha': 0.4687380486697353,
 'colsample_bytree': 0.8434345456331338,
 'subsample': 0.8991048362147761,
 'learning_rate': 0.04635349901133093,
 'max_depth': 21,
 'min_child_weight': 4,
 'num_class': 10,
 'eval_metric': 'merror'}

xgb_clf = xgb.train(xgb_params, dtrain, 700, 
                    watchlist, 
                    #early_stopping_rounds=40, 
                    verbose_eval=40)

[0]	eval-merror:0.109762	train-merror:0.060847
[40]	eval-merror:0.039048	train-merror:0.003968
[80]	eval-merror:0.035714	train-merror:0.000979
[120]	eval-merror:0.033095	train-merror:0.000212
[160]	eval-merror:0.031667	train-merror:0.000106
[200]	eval-merror:0.030476	train-merror:2.6e-05
[240]	eval-merror:0.029048	train-merror:0
[280]	eval-merror:0.028571	train-merror:0
[320]	eval-merror:0.027381	train-merror:0
[360]	eval-merror:0.027381	train-merror:0
[400]	eval-merror:0.026905	train-merror:0
[440]	eval-merror:0.02619	train-merror:0
[480]	eval-merror:0.025952	train-merror:0
[520]	eval-merror:0.02619	train-merror:0
[560]	eval-merror:0.02619	train-merror:0
[600]	eval-merror:0.02619	train-merror:0
[640]	eval-merror:0.025714	train-merror:0
[680]	eval-merror:0.025952	train-merror:0
[699]	eval-merror:0.025714	train-merror:0
CPU times: user 4min 51s, sys: 751 ms, total: 4min 51s
Wall time: 3min 12s


In [8]:
res_prob = xgb_clf.predict(xgb.DMatrix(test[col]))


In [9]:
res_prob

array([[1.5330388e-05, 1.0337118e-06, 9.9993300e-01, ..., 6.7331880e-06,
        1.3799670e-05, 6.9033999e-06],
       [9.9992967e-01, 3.7085936e-06, 1.7539347e-05, ..., 5.6655645e-06,
        2.6663158e-06, 2.0215375e-06],
       [2.2166570e-05, 6.6700894e-05, 5.9395569e-04, ..., 4.5025744e-04,
        7.3110173e-04, 9.9635822e-01],
       ...,
       [9.8342673e-07, 2.2822678e-06, 2.0961588e-06, ..., 1.2996718e-06,
        1.1538665e-05, 5.0148615e-06],
       [8.2418483e-05, 3.8576063e-06, 2.6976163e-06, ..., 2.5091669e-05,
        1.3010424e-05, 9.9912781e-01],
       [6.7546475e-06, 1.2771914e-06, 9.9987924e-01, ..., 4.9025157e-06,
        1.2602685e-05, 5.0466013e-05]], dtype=float32)

In [10]:
np.argmax(res_prob, axis=1)

array([2, 0, 9, ..., 3, 9, 2])

In [11]:
val_preds = np.argmax(xgb_clf.predict(dval), axis=1)

In [12]:
val_preds

array([8, 1, 9, ..., 7, 7, 5])

In [13]:
accuracy_score(Y_val, val_preds)

0.9742857142857143

In [14]:
res = np.argmax(res_prob, axis=1)

In [15]:
res.astype('int')

array([2, 0, 9, ..., 3, 9, 2])

In [16]:
sample_submission['Label'] = res.astype('int')
sample_submission.to_csv('submission.csv', index=False)
np.save('res_prob', res_prob)